In [26]:
# !pip install transformers -q
# %pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 262.6 kB/s eta 0:00:06
   - -------------------------------------- 0.0/1.5 MB 245.8 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.5 MB 416.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.5 MB 568.9 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.5 MB 765.3 kB/s eta 0:00:02
   ---------- ----------------------------- 0.4/1.5 MB 1.2 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.5 MB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 1.6 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 2.1 MB/s eta 0:00:01
   ---------------------------

In [1]:
from nltk.corpus import stopwords
from transformers import BertModel, BertTokenizer
import torch
import pandas as pd
from PyPDF2 import PdfReader

c:\Abhiraj\NLPProject\abhi_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
print(torch.__version__)

2.3.0+cpu


In [11]:
# nltk.download('stopwords')

In [2]:
class PdfParser:
    def __init__(self, filepath):
        self.filepath = filepath

    def extract_text(self) -> str:
        reader = PdfReader(self.filepath) 
        text = ''.join(page.extract_text() for page in reader.pages)
        return text

In [3]:
if __name__ == "__main__":
    extractor = PdfParser(r"C:\Abhiraj\InvoiceParser\Invoices\ALOIS TECH PVT LTD -14.pdf")
    extracted_text = extractor.extract_text()
    print(extracted_text)

#### Initialize stop words, Load the model and tokenizer

In [55]:
stop_words = set(stopwords.words('english'))
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [56]:
print(stop_words)

{'yourselves', 'this', 'these', 'those', 'being', 'your', 'itself', 'to', 'or', 'than', 'further', 'into', 'of', 'with', 'her', "isn't", 'weren', "that'll", "she's", "mightn't", 'more', 'off', 'whom', 'here', 'isn', 'are', 'hadn', "you're", 'couldn', 'that', 'so', 'in', 'ourselves', 'now', 'he', 'down', 'while', 'ma', 'have', 'were', 'does', 'been', 'from', 'my', 'then', 'own', "you'll", "it's", 'if', 'very', 'their', 'won', 'what', 'below', 're', 'through', "you'd", "shan't", 'over', 'not', 'hasn', 'only', 'ours', 'which', 'such', 'be', "wasn't", 'me', 'we', "should've", "weren't", "hadn't", 'will', 'should', 't', 'them', 'against', 'why', 'between', 'when', 'once', 'how', 'an', "couldn't", 'after', 'has', 'haven', 'i', 'ain', 'they', 'until', "doesn't", 'herself', 'on', 'its', 'aren', "aren't", 'll', 'theirs', 'she', 'did', "you've", 'each', 'doesn', 'as', "needn't", 'under', 'the', 'doing', 'there', 'mightn', 'up', 'was', 'didn', 'and', "didn't", 'who', "haven't", 'you', 'don', 'but

In [51]:
# corpus = ["Hello! I am a good boy. My name is Montu Sharma.", "I am a MachineLearning Engineer."]
corpus = [extracted_text]

In [52]:
def process_corpus(corpus, stop_words):   

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    stop_words = set(stopwords.words('english'))
    stop_words = stop_words

    processed_corpus = []

    for sentence in corpus:
        tokens = tokenizer.tokenize(sentence)
        # Remove stop words
        filtered_tokens = [token for token in tokens if token not in stop_words]
        # Prepend [CLS] and append [SEP]
        special_tokens = ['[CLS]'] + filtered_tokens + ['[SEP]']  
        # Encode the special tokens
        token_ids = tokenizer.convert_tokens_to_ids(special_tokens)        

        input_dict = {
            'input_ids': token_ids,
            'attention_mask': [1]*len(token_ids)
        }        

        # Set maximum length
        padded_sentence = tokenizer.pad(input_dict, padding='max_length', max_length=30)        

        attention_mask = [1 if i!= tokenizer.pad_token_id else 0 for i in padded_sentence['input_ids']]        
        
        # Append the padded sentence and its attention mask to processed_corpus
        processed_corpus.append((padded_sentence['input_ids'], attention_mask))  

    return processed_corpus

In [53]:
processed_corpus = process_corpus(corpus, stop_words)

# Display
for token_ids, attention_mask in processed_corpus:
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    
    data = {"tokens": tokens, 
            "token_ids": token_ids, 
            "attention_mask": attention_mask}
    
    print(f"\nTokens: {tokens}, \nToken IDs: {token_ids}, \nAttention Mask: {attention_mask}")
    
    df = pd.DataFrame(list(zip(tokens, token_ids, attention_mask)), columns=['Token', 'IDs', 'Attention Mask'])


Tokens: ['[CLS]', 'son', '##i', 'j', '##ha', 'business', 'analyst', '(', ')', '+', '91', '99', '##6', '##7', '##17', '##9', '##80', '##0', 'mumbai', '(', 'maharashtra', ')', 'em', '##ai', 'l', ':', 'son', '##ij', '##6', '##19', '##2', '@', 'gma', '##il', '.', 'com', 'profile', '•', 'techno', 'financial', 'management', 'professional', 'experience', '1', '0', '+', 'years', 'business', '/', 'system', 'analysis', ',', 'regulatory', 'reporting', ',', 'project', 'management', ',', 'requirement', 'gathering', ',', 'preparing', 'use', 'case', ',', 'test', 'case', ',', 'def', '##er', '##ral', 'list', ',', 'br', '##d', 'fr', '##d', '.', 'extensive', 'experience', 'client', 'facing', 'requirements', 'gathering', 'capture', 'process', 'map', 'process', 'vi', 'si', '##on', '.', 'def', '##t', 'design', 'review', 'various', 'documents', 'like', 'br', '##d', ',', 'fr', '##d', ',', 'use', 'case', 'specifications', ',', 'user', 'stories', ',', 'product', 'back', '##log', ',', 'sprint', 'back', '##log',

In [54]:
df

,Token,IDs,Attention Mask
0,[CLS],101,1
1,son,2365,1
2,##i,2072,1
3,j,1046,1
4,##ha,3270,1
...,...,...,...
1336,home,2188,1
1337,tool,6994,1
1338,sql,29296,1
1339,##m,2213,1


In [25]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [26]:
output = model(token_ids, attention_mask = attention_mask)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4546,  0.4363, -0.3296,  ..., -0.3637,  0.4343, -0.0601],
         [ 0.5521,  0.2751, -0.7303,  ..., -0.0442,  0.2307, -0.0526],
         [ 0.6285,  0.1741,  0.2261,  ...,  0.1098,  0.1802, -1.2926],
         ...,
         [ 0.1009, -0.0152, -0.0759,  ..., -0.1650, -0.3457, -0.1263],
         [ 0.0305, -0.2906,  0.0715,  ..., -0.2525, -0.3476,  0.1595],
         [ 0.3015,  0.2462,  0.0686,  ..., -0.1453, -0.3376, -0.2336]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.9604, -0.6381, -0.9947,  0.9564,  0.9562, -0.3733,  0.9847,  0.4575,
         -0.9756, -1.0000, -0.9169,  0.9966,  0.9572,  0.8458,  0.9297, -0.9705,
         -0.8064, -0.7094,  0.5099, -0.8736,  0.8075,  1.0000, -0.5156,  0.6334,
          0.7041,  0.9997, -0.9344,  0.9091,  0.9654,  0.8106, -0.8785,  0.5455,
         -0.9869, -0.4690, -0.9929, -0.9956,  0.7480, -0.8572, -0.2594, -0.3871,
         -0.8785,  0.6669,  1.00

In [79]:
# # Filter stop words and encode each sentence separately
# encoded_corpus = []
# for w in corpus:
#     # Tokenize the sentence
#     tokens = tokenizer.tokenize(w)
    
#     # Filter out stop words
#     filtered_tokens = [token for token in tokens if token not in stop_words]
    
#     # Encode the filtered tokens
#     encoded_sentence = tokenizer.convert_tokens_to_ids(filtered_tokens)
#     encoded_corpus.append(encoded_sentence)
# encoded_corpus

In [80]:
# tokens = tokenizer.encode(corpus, max_length=15, padding='max_length')
# tokens

In [ ]:
# names = tokenizer.convert_ids_to_tokens(encoded_corpus)

In [ ]:
# filtered_tokens = ['[CLS]'] + filtered_tokens + ['[SEP]']
# print(filtered_tokens)
# len(filtered_tokens)


['[CLS]', 'hello', '!', 'good', 'boy', '.', 'name', 'mont', '##u', 'sharma', '[SEP]']


11

In [ ]:
# filtered_tokens = filtered_tokens + ['[PAD]']
# len(filtered_tokens)

12

In [ ]:
# attention_mask = [1 if i != '[PAD]' else 0 for i in filtered_tokens]
# attention_mask

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

In [ ]:
# df = pd.DataFrame(list(zip(filtered_tokens, token_ids)), columns=['Token', 'IDs'])
# df